In [1]:
import sqlite3

In [2]:
with sqlite3.connect("scrabble.db") as conn:
    conn.execute("""
        CREATE TABLE players (
            player_id INTEGER PRIMARY KEY,
            name TEXT NOT NULL
        )
    """)

In [3]:
with sqlite3.connect("scrabble.db") as conn:
    conn.execute("""
        CREATE TABLE games (
            game_id INTEGER PRIMARY KEY,
            started_by INTEGER,
            winner INTEGER,
            FOREIGN KEY (started_by) REFERENCES players(player_id),
            FOREIGN KEY (winner) REFERENCES players(player_id)
        )
    """)

In [4]:
with sqlite3.connect("scrabble.db") as conn:
    conn.execute("""
        CREATE TABLE game_scores (
            game_id INTEGER,
            player_id INTEGER,
            score INTEGER,
            PRIMARY KEY (game_id, player_id),
            FOREIGN KEY (game_id) REFERENCES games(game_id),
            FOREIGN KEY (player_id) REFERENCES players(player_id)
        )
    """)

In [5]:
with sqlite3.connect("scrabble.db") as conn:
    conn.execute("""
    CREATE TABLE game_moves (
        game_id INTEGER,
        player_id INTEGER,
        move_number INTEGER,
        points INTEGER,
        PRIMARY KEY (game_id, player_id, move_number),
        FOREIGN KEY (game_id) REFERENCES games(game_id),
        FOREIGN KEY (player_id) REFERENCES players(player_id)
    )
    """)

In [6]:
with sqlite3.connect("scrabble.db") as conn:
    conn.execute("""
        INSERT INTO players (name)
        VALUES ('Deniz'), ('Danyel'), ('Robin')
    """)

In [7]:
def get_player_id(connection, name: str) -> int | None:
    response = connection.execute("""
        SELECT player_id
        FROM players
        WHERE name = ?
    """, (name,))

    result = response.fetchone()
    if result:
        return result[0]

    return None

In [8]:
import pandas as pd

df = pd.read_csv("scrabble_data.csv")

with sqlite3.connect("scrabble.db") as conn:
    deniz_id = get_player_id(conn, "Deniz")
    danyel_id = get_player_id(conn, "Danyel")
    robin_id = get_player_id(conn, "Robin")

with sqlite3.connect("scrabble.db") as conn:
    for _, row in df.iterrows():
        started_by_id = get_player_id(conn, row["starting_player"])
        winner_id = get_player_id(conn, row["winner"])

        conn.execute("""
            INSERT INTO games (started_by, winner)
            VALUES (?, ?)
        """, (started_by_id, winner_id))

In [11]:
with sqlite3.connect("scrabble.db") as conn:
    conn.row_factory = sqlite3.Row
    response = conn.execute("""
        SELECT g.game_id, p.name
        FROM games g
        INNER JOIN players p ON p.player_id = g.winner
    """)

    result = response.fetchall()
    print(result)


[(1, 'Robin'), (2, 'Robin'), (3, 'Deniz'), (4, 'Robin'), (5, 'Danyel'), (6, 'Danyel'), (7, 'Deniz'), (8, 'Robin'), (9, 'Robin'), (10, 'Danyel'), (11, 'Danyel'), (12, 'Robin'), (13, 'Danyel'), (14, 'Danyel'), (15, 'Deniz'), (16, 'Danyel'), (17, 'Deniz'), (18, 'Robin'), (19, 'Robin'), (20, 'Robin'), (21, 'Deniz'), (22, 'Deniz'), (23, 'Robin'), (24, 'Robin'), (25, 'Danyel'), (26, 'Danyel'), (27, 'Robin'), (28, 'Robin'), (29, 'Robin'), (30, 'Danyel'), (31, 'Robin'), (32, 'Danyel'), (33, 'Robin'), (34, 'Danyel'), (35, 'Deniz'), (37, 'Robin'), (38, 'Danyel'), (39, 'Danyel'), (40, 'Danyel'), (41, 'Deniz'), (42, 'Robin'), (43, 'Deniz'), (44, 'Deniz'), (45, 'Danyel'), (46, 'Deniz'), (47, 'Deniz'), (48, 'Robin'), (49, 'Danyel'), (50, 'Deniz'), (51, 'Danyel'), (52, 'Danyel'), (53, 'Robin'), (54, 'Danyel'), (55, 'Danyel'), (56, 'Robin'), (57, 'Deniz'), (58, 'Robin'), (59, 'Danyel'), (60, 'Deniz'), (61, 'Robin'), (62, 'Robin'), (63, 'Deniz'), (64, 'Deniz'), (65, 'Danyel'), (66, 'Danyel'), (67, 'Den

In [14]:
with sqlite3.connect("scrabble.db") as conn:
    conn.row_factory = sqlite3.Row
    response = conn.execute("""
        SELECT g.game_id, p.name
        FROM games g
        LEFT JOIN players p ON p.player_id = g.winner
    """)

    result = [dict(row) for row in response.fetchall()]
    print(result)


[{'game_id': 1, 'name': 'Robin'}, {'game_id': 2, 'name': 'Robin'}, {'game_id': 3, 'name': 'Deniz'}, {'game_id': 4, 'name': 'Robin'}, {'game_id': 5, 'name': 'Danyel'}, {'game_id': 6, 'name': 'Danyel'}, {'game_id': 7, 'name': 'Deniz'}, {'game_id': 8, 'name': 'Robin'}, {'game_id': 9, 'name': 'Robin'}, {'game_id': 10, 'name': 'Danyel'}, {'game_id': 11, 'name': 'Danyel'}, {'game_id': 12, 'name': 'Robin'}, {'game_id': 13, 'name': 'Danyel'}, {'game_id': 14, 'name': 'Danyel'}, {'game_id': 15, 'name': 'Deniz'}, {'game_id': 16, 'name': 'Danyel'}, {'game_id': 17, 'name': 'Deniz'}, {'game_id': 18, 'name': 'Robin'}, {'game_id': 19, 'name': 'Robin'}, {'game_id': 20, 'name': 'Robin'}, {'game_id': 21, 'name': 'Deniz'}, {'game_id': 22, 'name': 'Deniz'}, {'game_id': 23, 'name': 'Robin'}, {'game_id': 24, 'name': 'Robin'}, {'game_id': 25, 'name': 'Danyel'}, {'game_id': 26, 'name': 'Danyel'}, {'game_id': 27, 'name': 'Robin'}, {'game_id': 28, 'name': 'Robin'}, {'game_id': 29, 'name': 'Robin'}, {'game_id': 3

1. Table of all games (scores of each player, start player)
    1a. Moves of a game (evolution of points over time)
2. Stats for each player (best game, worst game, average points, ...)